In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-14 20:08:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.98MB/s    in 0.2s    

2022-10-14 20:08:50 (5.98 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [11]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

### Create DataFrames to match tables

In [12]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [13]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     176073|             1|
|   20992492|             1|
|    1397686|             1|
|   50371142|             1|
|     235096|             1|
|   12320806|             1|
|   19707203|             1|
|   15207172|             1|
|     538703|             1|
|   11223135|             1|
|   31447151|             1|
|   35961247|             1|
|   17835519|             1|
|   15404885|             1|
|   10182610|             1|
|   52399092|             1|
|   17133428|             1|
|   27870423|             1|
|   11574168|             1|
|   30685607|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0594450209|Barnes & Noble 7"...|
|0594549531|Barnes & Noble Gr...|
|0594549558|Barnes & Noble Jo...|
|089933623X|New 7" Inch Tft-l...|
|1616831022|Barnes & Noble Ly...|
|7514302255|Smile Case Full P...|
|7539945370|Smile Case Full P...|
|7810294245|Dekcell PDA Batte...|
|8477599297|Zoomdog Z5 Portab...|
|9573455226|Nokia 2.5mm Male ...|
|9621404312|Amazing Gag Novel...|
|9745126519|Toyota Hilux Vigo...|
|9967120169|PRO Series Lenovo...|
|9984983978|Professional Retr...|
|9985606051|Sansa Fuze+ (Medi...|
|9985776097|Premium Accessory...|
|B00005BVA9|SpongeBob SquareP...|
|B00006B8MK|Kensington 56146 ...|
|B00006HOOM|Yamaha NS-5290 2-...|
|B00006I566|COBY CX-CD330 - C...|
+----------+--------------------+
only showing top 20 rows



In [15]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8MEA6IGAHO0B|   20422322|B00MC4CED8|     217304173| 2015-08-31|
|R31LOQ8JGLPRLK|   40835037|B00OQMFG1Q|     137313254| 2015-08-31|
|R2Y0MM9YE6OP3P|   51469641|B00QERR5CY|      82850235| 2015-08-31|
| RRB9C05HDOD4O|    4332923|B00QUFTPV4|     221169481| 2015-08-31|
|R26I2RI1GFV8QG|   44855305|B0067XVNTG|     563475445| 2015-08-31|
| RY8DDL22YG4R5|    7846966|B00KA6CCVY|     713636156| 2015-08-31|
|R2AT2426ZHFUHH|   21299354|B00MJCDPM2|     754171872| 2015-08-31|
|R3RRXU2R23NMQ9|   28902968|B00ET5AWBY|     508373185| 2015-08-31|
|R250PR8VJUZ62F|    2074438|B00Z9K379C|     759387370| 2015-08-31|
| RBEMQ29WJBHYG|   48701722|B00YO3UYXW|     913911909| 2015-08-31|
|R19VVIUT4BZCMT|    2990247|B011W26BU2|     938399424| 2015-08-31|
|R1DT8JJUQHUKUL|   52946117|B00H8PI78C|     661515294| 2015-08

In [16]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)